# Probando conexion con la base de datos PostgreSQL local

In [9]:
pip install python-dotenv


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 55.7 MB/s  0:00:00

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import psycopg2
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Fetch variables
USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")

# Connect to the database
try:
    connection = psycopg2.connect(
        user=USER,
        password=PASSWORD,
        host=HOST,
        port=PORT,
        dbname=DBNAME
    )
    print("Connection successful!")
    
    # Create a cursor to execute SQL queries
    cursor = connection.cursor()
    
    # Example query
    cursor.execute("SELECT NOW();")
    result = cursor.fetchone()
    print("Current Time:", result)

    # Close the cursor and connection
    #cursor.close()
    #connection.close()
    #print("Connection closed.")

except Exception as e:
    print(f"Failed to connect: {e}")

Connection successful!
Current Time: (datetime.datetime(2025, 11, 3, 3, 35, 16, 182230, tzinfo=datetime.timezone.utc),)


## Listando tablas disponibles

In [14]:
cursor = connection.cursor()
cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
""")

tables = cursor.fetchall()
print("Tablas encontradas:")
for table in tables:
    print(table[0])

cursor.close()

Tablas encontradas:
n8n_chat_histories
airbnb_listings_final
geography_columns
geometry_columns
spatial_ref_sys
scraped_data
tweets_results
google_search
instagram_posts
trivago_hoteles
facebook_posts


## Listando campos de la tabla instagram_posts

In [15]:
table_name = 'instagram_posts'  

cursor = connection.cursor()
cursor.execute("""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = %s
""", (table_name,))

columns = cursor.fetchall()
print(f"Columnas en {table_name}:")
for column in columns:
    print(f"- {column[0]} ({column[1]})")

cursor.close()

Columnas en instagram_posts:
- created_at (timestamp with time zone)
- mentions (jsonb)
- engagement (jsonb)
- id (uuid)
- location (jsonb)
- hashtags (jsonb)
- post_id (text)
- username (text)
- caption (text)


# Seleccionamos la data

In [16]:
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Fetch variables
USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")

# Conexión a tu base de datos
connection = psycopg2.connect(
        user=USER,
        password=PASSWORD,
        host=HOST,
        port=PORT,
        dbname=DBNAME
    )

table_name = 'instagram_posts'
query = f"SELECT * FROM {table_name};"

df = pd.read_sql(query, connection)
df.to_csv("instagram_posts.csv", index=False, encoding='utf-8', quoting=1)
print(df.head())
connection.close()

/tmp/ipykernel_7043/4114259078.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


                                     id post_id           username  \
0  54e656fd-04cd-4c84-aa05-6f5dc977d0a0  IG_001  aventurero_andino   
1  d85d4f85-f5e5-4c1b-9cb6-d7344d44c955  IG_002  exploradora_selva   
2  308dc8dc-26c2-473f-b3df-17566b6cdb18  IG_003   playas_del_norte   
3  3c650525-c5be-4478-9c95-e2f334216a6e  IG_004     trekking_lover   
4  efaebd80-e219-4d74-b852-bffc8d89abe1  IG_005        foodie_lima   

                                             caption  \
0  ¡Increíble fin de semana en Cusco! Volamos con...   
1  Una semana completa en Iquitos, explorando el ...   
2  Disfrutando del sol en Máncora por 5 días. Tom...   
3  Ruta del Salkantay a Machu Picchu. Fueron 4 dí...   
4  Fin de semana gastronómico en Lima. Nos movimo...   

                       location                                 hashtags  \
0       {'name': 'Cusco, Peru'}  [Cusco, PeruTravel, MachuPicchu, Viaje]   
1   {'name': 'Iquitos, Loreto'}        [Iquitos, Amazonas, SelvaPeruana]   
2    {'name': 

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          100 non-null    object             
 1   post_id     100 non-null    object             
 2   username    100 non-null    object             
 3   caption     100 non-null    object             
 4   location    100 non-null    object             
 5   hashtags    100 non-null    object             
 6   mentions    100 non-null    object             
 7   engagement  100 non-null    object             
 8   created_at  100 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(8)
memory usage: 7.2+ KB


# Almacenamos los datos en HDFS

In [18]:
import subprocess

In [20]:
hdfs_path = "raw/instagram/instagram_posts.csv"

subprocess.run(["hdfs", "dfs", "-put", "-f", "instagram_posts.csv", hdfs_path], check=True)

print("Archivo cargado a HDFS:", hdfs_path)

Archivo cargado a HDFS: raw/instagram/instagram_posts.csv


# Procesamos los datos

In [21]:
# ==============================================================================
# 1. IMPORTACIONES E INICIO DE SESIÓN DE SPARK
# ==============================================================================
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, to_date, count, when
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, FloatType, TimestampType
import re

# Detiene cualquier sesión de Spark existente para asegurar una configuración limpia
try:
    spark.stop()
except NameError:
    pass

print("Configurando e iniciando la sesión de Spark...")
spark = SparkSession.builder \
    .appName("ETL_Instagram_HDFS") \
    .getOrCreate()
print("Sesión de Spark iniciada.")

Configurando e iniciando la sesión de Spark...


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/02 22:38:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Sesión de Spark iniciada.


In [31]:
# ==============================================================================
# 2. LECTURA DE DATOS DESDE HDFS
# ==============================================================================

# Definir esquema basado en las columnas reales de Instagram
schema = StructType([
    StructField("id", StringType(), True),
    StructField("post_id", StringType(), True),
    StructField("username", StringType(), True),
    StructField("caption", StringType(), True),
    StructField("location", StringType(), True),
    StructField("hashtags", StringType(), True),
    StructField("mentions", StringType(), True),
    StructField("engagement", StringType(), True),
    StructField("created_at", TimestampType(), True)
])

print("Leyendo datos desde HDFS...")
df_from_supabase = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiLine", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .option("encoding", "UTF-8") \
    .option("delimiter", ",") \
    .schema(schema) \
    .csv("hdfs://localhost:9000/user/user/raw/instagram/instagram_posts.csv")

print(f"Datos cargados desde HDFS. Se leyeron {df_from_supabase.count()} registros.")

Leyendo datos desde HDFS...
Datos cargados desde HDFS. Se leyeron 100 registros.


In [32]:
df_from_supabase

DataFrame[id: string, post_id: string, username: string, caption: string, location: string, hashtags: string, mentions: string, engagement: string, created_at: timestamp]

In [33]:
# ==============================================================================
# 3. DEFINICIÓN DE LÓGICA DE PROCESAMIENTO (UDFs)
# ==============================================================================
TRANSPORT_KEYWORDS = {'aéreo': ['vuelo', 'volamos', 'avión', 'latam', 'sky airline', 'jetsmart', 'star peru', 'avioneta'],'terrestre': ['bus', 'carro', 'auto', 'camioneta', 'cruz del sur', 'movil tours', 'en carretera', 'van'],'tren': ['tren', 'perurail', 'inca rail', 'andean explorer'],'marítimo': ['bote', 'lancha', 'barco', 'navegando']}
ACCOMMODATION_KEYWORDS = {'hotel': ['hotel', 'resort', 'lodge', 'boutique'],'hostal': ['hostal', 'hostel', 'hospedaje'],'otro': ['airbnb', 'casa', 'bungalow', 'cabaña', 'campamento', 'acampamos', 'camping']}
PERUVIAN_DESTINATIONS = ['cusco', 'machu picchu', 'arequipa', 'lima', 'iquitos', 'mancora', 'paracas', 'huaraz','nazca', 'puno', 'lago titicaca', 'trujillo', 'chiclayo', 'tarapoto', 'ayacucho','cajamarca', 'chachapoyas', 'kuelap', 'gocta', 'vinicunca', 'montaña de 7 colores','huacachina', 'ica', 'ollantaytambo', 'valle sagrado', 'urubamba', 'colca', 'salkantay','choquequirao', 'tambopata', 'pacaya samiria', 'lunahuaná']

def extract_entity(text, keywords_dict):
    if text is None: return None
    text_lower = text.lower()
    for category, keywords in keywords_dict.items():
        for keyword in keywords:
            if re.search(r'\b' + re.escape(keyword) + r'\b', text_lower):
                return category
    return None

def extract_duration(text):
    if text is None: return None
    text_lower = text.lower()
    match = re.search(r'(\d+)\s*(días|noches)', text_lower)
    if match: return f"{match.group(1)} días/noches"
    if 'una semana' in text_lower or 'semana entera' in text_lower: return "1 semana"
    if 'dos semanas' in text_lower: return "2 semanas"
    if 'fin de semana' in text_lower or 'finde' in text_lower: return "fin de semana"
    if 'full day' in text_lower or 'un día' in text_lower or 'medio día' in text_lower: return "1 día (Full Day)"
    return None

def extract_location_spark(location_struct, caption):
    if location_struct and hasattr(location_struct, 'name') and location_struct.name:
        location_name = location_struct.name.split(',')[0].strip()
        if location_name.lower() not in ['peru', 'cusco', 'lima'] or len(location_struct.name.split(',')) == 1:
            return location_name
    if caption:
        text_lower = caption.lower()
        for dest in PERUVIAN_DESTINATIONS:
            if dest in text_lower:
                return dest.title()
    if location_struct and hasattr(location_struct, 'name') and location_struct.name:
        return location_struct.name.split(',')[0].strip()
    return None

extract_transport_udf = udf(lambda caption: extract_entity(caption, TRANSPORT_KEYWORDS), StringType())
extract_accommodation_udf = udf(lambda caption: extract_entity(caption, ACCOMMODATION_KEYWORDS), StringType())
extract_duration_udf = udf(extract_duration, StringType())
extract_location_udf = udf(extract_location_spark, StringType())

In [34]:
# ==============================================================================
# 4. APLICACIÓN DE TRANSFORMACIONES
# ==============================================================================
print("Aplicando transformaciones al DataFrame...")

df_processed = df_from_supabase.withColumn("lugar_viaje", extract_location_udf(col("location"), col("caption"))) \
                           .withColumn("fecha", to_date(col("created_at"))) \
                           .withColumn("medio_transporte", extract_transport_udf(col("caption"))) \
                           .withColumn("duracion", extract_duration_udf(col("caption"))) \
                           .withColumn("alojamiento", extract_accommodation_udf(col("caption")))

df_final = df_processed.select("lugar_viaje", "fecha", "medio_transporte", "duracion", "alojamiento")

Aplicando transformaciones al DataFrame...


In [35]:
# ==============================================================================
# 5. MOSTRAR RESULTADOS Y GUARDAR EN HDFS
# ==============================================================================
print("\n--- Vista Previa del Dataset Procesado ---")
df_final.show(15, truncate=False)

print("\n--- Resumen de Valores Nulos por Columna ---")
df_final.select([count(when(col(c).isNull(), c)).alias(c) for c in df_final.columns]).show()


--- Vista Previa del Dataset Procesado ---
+------------+----------+----------------+----------------+-----------+
|lugar_viaje |fecha     |medio_transporte|duracion        |alojamiento|
+------------+----------+----------------+----------------+-----------+
|Cusco       |2025-05-22|aéreo           |3 días/noches   |hotel      |
|Iquitos     |2025-08-10|aéreo           |1 semana        |hotel      |
|Lima        |2025-02-15|terrestre       |5 días/noches   |otro       |
|Cusco       |2025-09-05|tren            |4 días/noches   |NULL       |
|Lima        |2025-03-28|NULL            |fin de semana   |otro       |
|Arequipa    |2025-07-19|aéreo           |3 días/noches   |hotel      |
|Lima        |2025-04-12|terrestre       |2 días/noches   |hostal     |
|Arequipa    |2025-08-25|terrestre       |3 días/noches   |NULL       |
|Cusco       |2025-07-27|aéreo           |1 semana        |otro       |
|Cusco       |2025-09-18|NULL            |1 día (Full Day)|NULL       |
|Lima        |2025-0

In [36]:
# Guardamos los datos procesados en HDFS
hdfs_path_csv = "hdfs://localhost:9000/user/user/processed/instagram/instagram_posts_clean.csv"

df_final.write \
    .option("header", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .option("multiLine", "true") \
    .option("encoding", "UTF-8") \
    .option("delimiter", ",") \
    .mode("overwrite") \
    .csv(hdfs_path_csv)

print(f"Datos guardados como CSV en HDFS en la ruta: {hdfs_path_csv}")

Datos guardados como CSV en HDFS en la ruta: hdfs://localhost:9000/user/user/processed/instagram/instagram_posts_clean.csv


# Análisis de datos procesados

## Leyendo datos limpios desde HDFS

In [37]:
instagram_processed_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiLine", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .option("encoding", "UTF-8") \
    .option("delimiter", ",") \
    .csv("hdfs://localhost:9000/user/user/processed/instagram/instagram_posts_clean.csv")

instagram_processed_df.printSchema()
print("Filas:", instagram_processed_df.count())
instagram_processed_df.show(10)

root
 |-- lugar_viaje: string (nullable = true)
 |-- fecha: date (nullable = true)
 |-- medio_transporte: string (nullable = true)
 |-- duracion: string (nullable = true)
 |-- alojamiento: string (nullable = true)

Filas: 100
+-----------+----------+----------------+----------------+-----------+
|lugar_viaje|     fecha|medio_transporte|        duracion|alojamiento|
+-----------+----------+----------------+----------------+-----------+
|      Cusco|2025-05-22|           aéreo|   3 días/noches|      hotel|
|    Iquitos|2025-08-10|           aéreo|        1 semana|      hotel|
|       Lima|2025-02-15|       terrestre|   5 días/noches|       otro|
|      Cusco|2025-09-05|            tren|   4 días/noches|       NULL|
|       Lima|2025-03-28|            NULL|   fin de semana|       otro|
|   Arequipa|2025-07-19|           aéreo|   3 días/noches|      hotel|
|       Lima|2025-04-12|       terrestre|   2 días/noches|     hostal|
|   Arequipa|2025-08-25|       terrestre|   3 días/noches|      

# Deteniendo sesión de Spark

In [38]:
# ==============================================================================
# 6. DETENER LA SESIÓN DE SPARK
# ==============================================================================
spark.stop()
print("Proceso completado. Sesión de Spark detenida.")

Proceso completado. Sesión de Spark detenida.
